In [2]:
import random
from scipy import optimize

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Weekly Exercises: week 7

### Calculate the avalanche duration probability $P_>(t)$ if $λ_i = λ$ for all $t$ and all neurons. This leads, as only sketched in class, to the following integral, which can be solved through the saddle point approximation: 

$$
\gamma \int_0^\infty d\lambda\; e^{-\gamma\lambda}(1-e^{-\delta\lambda})^n
$$


Let's start from $P_>(t|\lambda_1,\lambda_2,...,\lambda_n)$, the probability of having an avalanche of duration longer than $t=n\,dt$ given the firing rates $\lambda_i=r(t_i)$, which can be computed recursively:

$$
P_>(t|\lambda_1,\lambda_2,...,\lambda_n)=\prod_{i=1}^n[1-(1-\lambda_i dt)^N]
$$
where $N$ is the number of neurons and $dt=\delta/N$, so then $n=Nt/\delta$.

if we assume that the $\lambda_i$ are independent and that $\lambda_i=\lambda \;\; \forall i$, we can sample from a distribution $\lambda \sim Q(\lambda)$ and marginalizing $P_>(t|\lambda_1,\lambda_2,...,\lambda_n)$ we have:

$$
P_>(t) = \int_0^\infty Q(\lambda)\;\left[1-\left(1-\frac{\delta}{N}\,\lambda\right)^N\right]^n \;d\lambda \\
=\int_0^\infty Q(\lambda)\;\left[1-e^{-\delta \lambda}\right]^n \;d\lambda \\
=\gamma \int_0^\infty e^{-\gamma\lambda}\;\left[1-e^{-\delta \lambda}\right]^n \;d\lambda
$$

where: $$lim_{N \rightarrow \infty}\left(1-\frac{\delta}{N}\,\lambda\right)^N=e^{-\delta \lambda}$$ and: $$Q(\lambda)=\gamma\,e^{-\gamma\lambda}$$

In particular we can rearrange $P_>(t)$ and apply the saddle point approximation when $n\rightarrow\infty$:

$$
P_>(t) = \gamma \int_0^\infty d\lambda\; e^{-\gamma\lambda}(1-e^{-\delta\lambda})^n \\
= \gamma \int_0^\infty d\lambda\; e^{n\left[-\frac{\gamma}{n}\lambda+ln\left( 1-e^{-\delta\lambda} \right) \right]} \\
= \gamma \int_0^\infty d\lambda\; e^{n f(\lambda)}  \\
\sim \gamma \;\frac{\sqrt{2π}\; e^{nf(\bar{\lambda})}}{\sqrt{|\,n\;f''(\bar{\lambda})\,|}}
$$

where $\bar{\lambda}$ is the global maximum of the function $f(\lambda)=-\frac{\gamma}{n}\lambda+ln\left( 1-e^{-\delta\lambda} \right)$ in the $(0,\infty)$ interval. 
So then we want to find this global maximum $\bar{\lambda}$ that is given by:

$$
f'(\lambda)=0 \\
-\frac{\gamma}{n}+\frac{ \delta e^{-\delta\bar{\lambda}}}{ 1-e^{-\delta\bar{\lambda}}}=0 \\
\gamma=(\gamma+\delta\,n)\;e^{-\delta \bar{\lambda}} \\
\bar{\lambda}=\frac{1}{\delta}\;ln\left( 1+ \frac{\delta \,n}{\gamma} \right)
\\
\Rightarrow f(\bar{\lambda})=-\frac{\gamma}{n\,\delta}\;ln\left( 1+ \frac{\delta \,n}{\gamma} \right)+ln\left( 1-\frac{\gamma}{\gamma+\delta \,n} \right)
= -\frac{\gamma}{n\,\delta}\;ln\left( 1+\frac{\delta \,n}{\gamma} \right)-ln\left( 1+\frac{\gamma}{\delta \,n} \right)
\\
\Rightarrow f''(\bar{\lambda})=\frac{-\delta^2 \;e^{-\delta\bar{\lambda}}}{(1-e^{-\delta\bar{\lambda}})^2}=-\gamma\;\frac{\gamma+\delta\,n}{n^2}=-\frac{\gamma\,\delta}{n}\left(\frac{\gamma}{n\,\delta}+1\right)<0
$$





Finally through the saddle point approxiation we obtain:
$$
P_>(t) = \sqrt{\frac{2\pi\gamma}{\delta}}\;\left( 1+\frac{\delta\,n}{\gamma}\right)^{-\gamma/\delta}\;\left( 1+\frac{\gamma}{\delta\,n}\right)^{-n-1/2}
$$
where $\left( 1+\frac{\delta\,n}{\gamma}\right)^{-\gamma/\delta}$ is the dominant term as $n\rightarrow\infty$.

### **Optional**. Create a time series $λ(t)$, with $t=1,2,...,T$ where at each time $t$, the value of $λ$ is extracted from an exponential distribution. Then simulate $N=100$ independent heterogeneous Poisson processes, where each one describes the spikes events of a single neuron, but all have the same time dependent rate parameter $λ(t)$.

In order to simulate a heterogeneous Poisson process we can use the rejection sampling technique.

**1)**  Generate a spike train as a homogeneous Poisson process with maximum spike frequency $\lambda_{max}$, defined as:

$$
\lambda_{max}\ge \lambda(t) \;\;\; \forall t \in \left[ 0,T \right] 
$$


**2)** Discard some samples to locally reduce the spike maximum rate $\lambda_{max}$ according to the thinning procedure and bring it back to $\lambda(t)$. In this way we will obtain a Posson process that will evolve compatibly with $\lambda(t)$: a heterogeneous Poisson processes. 

In the homogeneous Poisson process we have:

- $P_T(n)$, the probability that any sequence of $n$ spikes occurs in the time interval $T$, that is a Poisson distribution:

$$
P_T(n) = \frac{(\lambda_{max}T)^n}{n!} e^{-\lambda_{max}T}
$$

- The interspike interval distribution $p(\tau \le t_{i+1}-t_i<\tau+\Delta t)$, the probability density of time intervals between two adjacent spikes, is an exponential probability density function:

$$
p(\tau \le t_{i+1}-t_i<\tau+\Delta t) = \lambda_{max} e^{-\lambda_{max}\tau}
$$


To summarize, the number of events in a given time interval $P_T(n)$ is modeled using a discrete Poisson distribution and, thanks to the properties of the Poisson distribution, the interval of time between consecutive events $p(\tau \le t_{i+1}-t_i<\tau+\Delta t)$ is modeled using an exponential distribution.

In [3]:
def NHPP(max_l, max_t, rs):
  t_series = []
  t = 0
  while True:
    t += random.expovariate(max_l)
    if t > max_t:
        break
    idx = np.where(np.arange(max_t) < t)[0][-1]
    if random.random() <= rs[idx]/max_l:
      t_series.append(t)
  return t_series

In [4]:
T = 80
gamma = 1
rates = np.random.exponential(1./gamma, T)   
max_rate = np.max(rates)

### Visualization of different realizations - 5 Neurons

In [5]:
N_test = 5

In [6]:
fig = make_subplots(rows = N_test, cols = 1, shared_xaxes = True, x_title = 'Time', y_title = 'Spikes')   
for i in range(N_test):
  test_train = NHPP(max_rate, T, rates)
  trace = go.Bar(x = test_train,
                 y = np.ones((len(test_train))),
                 width = len(test_train)*[0.03],
                 marker_color = 'darkred',
                 hovertemplate = 'time = %{x}<extra></extra>')
  fig.add_trace(trace, row = i+1, col = 1)
fig.update_layout(title_text = 'Different realizations of a heterogeneous Poisson process - Spike Trains', showlegend = False)
fig.update_yaxes(showticklabels = False)
fig.show()

### Visualization of different realizations - 100 Neurons

In [7]:
N = 100

In [8]:
fig = make_subplots(rows = N, cols = 1, shared_xaxes = True, x_title = 'Time', y_title = 'Spikes')   
for i in range(N):
  test_train = NHPP(max_rate, T, rates)
  trace = go.Bar(x = test_train,
                 y = np.ones((len(test_train))),
                 width = len(test_train)*[0.03],
                 marker_color = 'darkred',
                 hovertemplate = 'time = %{x}<extra></extra>')
  fig.add_trace(trace, row = i+1, col = 1)
fig.update_layout(title_text = 'Different realizations of a heterogeneous Poisson process - Spike Trains', showlegend = False)
fig.update_yaxes(showticklabels = False)
fig.show()

In [9]:
neuro_spikes = np.empty((0))
for i in range(N):
  neuro_spikes = np.append(neuro_spikes, NHPP(max_rate, T, rates))

counts, edges = np.histogram(neuro_spikes, bins = 'sqrt') 
centers = 0.5*(edges[:-1]+edges[1:])                                       
trace = go.Bar(x = centers,
              y = counts,
              width = np.diff(edges),
              marker_color = 'darkred',
              hovertemplate = '%{y}<extra></extra>')
fig = make_subplots(rows = 1, cols = 1)
fig.add_trace(trace)
fig.update_layout(title_text = 'Different realizations of a heterogeneous Poisson process - Spikes frequency in time',
                  xaxis = dict(title = 'Time', titlefont_size = 16),
                  yaxis = dict(title = 'Frequency', titlefont_size = 16))
fig.show()

The following plots compare the simulation of the heterogeneous Poisson process with the Poisson assumption. In particular the last plot shows that the interspike interval distribution is clearly not exponential as computed for the homogeneous Poisson process.

In [10]:
timediff_spikes = np.empty((0))
for i in range(N):
  timediff_spikes = np.append(timediff_spikes, np.diff(NHPP(max_rate, T, rates)))

counts, edges = np.histogram(timediff_spikes, bins = 'sqrt') 
centers = 0.5*(edges[:-1]+edges[1:])                                       
trace = go.Bar(x = centers,
              y = counts,
              width = np.diff(edges),
              hovertemplate = '%{y}<extra></extra>')
fig = make_subplots(rows = 1, cols = 1)
fig.add_trace(trace)
fig.update_layout(title_text = 'Different realizations of a heterogeneous Poisson process - Interspike interval frequency',
                  xaxis = dict(title = 'Interspike interval (time)', titlefont_size = 16),
                  yaxis = dict(title = 'Frequency', titlefont_size = 16))
fig.show()

In [11]:
def exp_fit(x, g):
    return g*np.exp(-g*x)

In [12]:
timediff_spikes = np.empty((0))
for i in range(N):
  timediff_spikes = np.append(timediff_spikes, np.diff(NHPP(max_rate, T, rates)))


counts, edges = np.histogram(timediff_spikes, bins = 'sqrt', density = True) 
centers = 0.5*(edges[:-1]+edges[1:])                                       
trace = go.Bar(x = centers,
              y = counts,
              width = np.diff(edges),
              name = 'Interspike interval distribution',
              hovertemplate = '%{y}<extra></extra>')
fig = make_subplots(rows = 1, cols = 1)
fig.add_trace(trace)

popt, pcov = optimize.curve_fit(exp_fit, centers, counts, p0 = [max_rate], method = "lm", full_output = False)
param = pd.DataFrame( np.dstack((popt, np.sqrt(np.diag(pcov)) )).reshape(1,2), columns = ["gamma", "std_gamma"])
trace = go.Scatter(x = centers, 
                   y = exp_fit(centers, *popt),
                   name = 'Exponential Interpolation',
                   mode = 'lines',
                   hovertemplate = '%{y}<extra></extra>')
fig.add_trace(trace)
fig.update_layout(title_text = 'Different realizations of a heterogeneous Poisson process - Interspike interval distribution vs. Exponential Interpolation',
                  xaxis = dict(title = 'Interspike interval (time)', titlefont_size = 16),
                  yaxis = dict(title = 'Frequency', titlefont_size = 16))

print("\n\nExponential interpolation parameters - Gamma: g = %.2f \u00B1 %.2f\n" % (param["gamma"], param["std_gamma"]))
tss = np.sum((np.mean(counts) - counts)**2)
ssr = np.sum((counts - exp_fit(centers, *popt))**2)
rsq = 1 - ssr / tss
print("Correlation coefficient: R^2 = %.4f" %rsq, "\n\t\t\t R = %.4f \n\n" %np.sqrt(rsq))
fig.show()



Exponential interpolation parameters - Gamma: g = 1.42 ± 0.04

Correlation coefficient: R^2 = 0.9495 
			 R = 0.9744 


